## Assignment 3 - CS20B021 - CS6910

### Importing Libraries

In [101]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os
import time
import math

import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Global Variables and Helper Functions

In [102]:
MAX_LEN = 40
SOS_token = 0
EOS_token = 1

def filterPair(p):
    return len(p[0]) < MAX_LEN and \
        len(p[1]) < MAX_LEN

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

### Data Preprocessing

In [103]:
# Load data
train_data = pd.read_csv('aksharantar_sampled/tel/tel_train.csv')
valid_data = pd.read_csv('aksharantar_sampled/tel/tel_valid.csv')
test_data = pd.read_csv('aksharantar_sampled/tel/tel_test.csv')

# Prepare data in pairs
train_pairs = []
for line in train_data.values:
    train_pairs.append([line[0], line[1]])
    
valid_pairs = []
for line in valid_data.values:
    valid_pairs.append([line[0], line[1]])

test_pairs = []
for line in test_data.values:
    test_pairs.append([line[0], line[1]])

# Filter pairs with length > MAX_LEN
train_pairs = filterPairs(train_pairs)
valid_pairs = filterPairs(valid_pairs)
test_pairs = filterPairs(test_pairs)

### Language Class

In [104]:
class Lang:
    def __init__(self, type):
        self.type = type
        self.char2index = {}
        self.char2count = {}
        self.index2char = {0: "SOS", 1: "EOS"}
        self.n_chars = 2  # Count SOS and EOS

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1

    def addWord(self, word):
        for char in word:
            self.addChar(char)

input_lang = Lang('input')
output_lang = Lang('output')

for pair in train_pairs:
    input_lang.addWord(pair[0])
    output_lang.addWord(pair[1])

def indexesFromWord(lang, word):
    return [lang.char2index[char] for char in word]

def tensorFromWord(lang, word):
    indexes = indexesFromWord(lang, word)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorFromPair(pair):
    input_tensor = tensorFromWord(input_lang, pair[0])
    target_tensor = tensorFromWord(output_lang, pair[1])
    return (input_tensor, target_tensor)

### Encoder Class

In [105]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size = 128, embed_size = 64, num_layers = 1, cell_type = 'gru', dropout = 0.1, bidirectional = False):
        super(Encoder, self).__init__()
        # Class Variables
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.bidirectional = bidirectional
        self.cell_type = cell_type

        # Layers and Cells Initialized with parameters
        self.embedding = nn.Embedding(input_size, embed_size)
        if cell_type == 'gru':
            self.cell = nn.GRU(input_size = self.embed_size, hidden_size = self.hidden_size, num_layers = self.num_layers, dropout = self.dropout, bidirectional = self.bidirectional)
        elif cell_type == 'rnn':
            self.cell = nn.RNN(input_size = self.embed_size, hidden_size = self.hidden_size, num_layers = self.num_layers, dropout = self.dropout, bidirectional = self.bidirectional)
        elif cell_type == 'lstm':
            self.cell = nn.LSTM(input_size = self.embed_size, hidden_size = self.hidden_size, num_layers = self.num_layers, dropout = self.dropout, bidirectional = self.bidirectional)
        else:
            raise ValueError('Invalid cell type specified')
        
    def forward(self, input, hidden, cell):
        embedded = self.embedding(input).view(1, 1, -1)

        # Forward pass through the cell
        if self.cell_type == 'lstm':
            output, (hidden, cell) = self.cell(embedded, (hidden, cell))
        else:
            output, hidden = self.cell(embedded, hidden)

        return output, hidden, cell
    
    def initHidden(self):
        # Initialize hidden state with zeros
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

### Decoder Class

In [106]:
class Decoder(nn.Module):
    def __init__(self, output_size,hidden_size = 128, embed_size = 64, num_layers = 1, cell_type = 'gru', dropout = 0.1, bidirectional = False):
        super(Decoder, self).__init__()
        # Class Variables
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.bidirectional = bidirectional
        self.cell_type = cell_type

        # Layers and Cells Initialized with parameters
        self.embedding = nn.Embedding(output_size, embed_size)
        if cell_type == 'gru':
            self.cell = nn.GRU(input_size = self.embed_size, hidden_size = self.hidden_size, num_layers = self.num_layers, dropout = self.dropout, bidirectional = self.bidirectional)
        elif cell_type == 'rnn':
            self.cell = nn.RNN(input_size = self.embed_size, hidden_size = self.hidden_size, num_layers = self.num_layers, dropout = self.dropout, bidirectional = self.bidirectional)
        elif cell_type == 'lstm':
            self.cell = nn.LSTM(input_size = self.embed_size, hidden_size = self.hidden_size, num_layers = self.num_layers, dropout = self.dropout, bidirectional = self.bidirectional)
        else:
            raise ValueError('Invalid cell type specified')
        self.out = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell):
        output = self.embedding(input).view(1, 1, -1)

        # Forward pass through the cell
        if self.cell_type == 'lstm':
            output, (hidden, cell) = self.cell(output, (hidden, cell))
        else:
            output, hidden = self.cell(output, hidden)

        # Output with softmax
        output = self.softmax(self.out(output[0]))
        return output, hidden, cell

    def initHidden(self):
        # Initialize hidden state with zeros
        return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)


### Attention Decoder 

### Sequence to Sequence Model

In [107]:
class Seq2Seq():
    def __init__(self, hidden_size = 128, embed_size = 64, num_layers = 1, cell_type = 'gru', dropout = 0.1, bidirectional = False, optimizer = 'sgd', lr = 0.01):
        
        # Class Variables
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.dropout = dropout
        self.bidirectional = bidirectional
        self.optimizer = optimizer
        self.lr = lr

        # Initialize Encoder and Decoder
        self.encoder = Encoder(input_lang.n_chars, hidden_size, embed_size, num_layers, cell_type, dropout, bidirectional).to(device)
        self.decoder = Decoder(output_lang.n_chars, hidden_size, embed_size, num_layers, cell_type, dropout, bidirectional).to(device)

        # Initialize Optimizer
        if optimizer == 'sgd':
            self.encoder_optimizer = optim.SGD(self.encoder.parameters(), lr = self.lr)
            self.decoder_optimizer = optim.SGD(self.decoder.parameters(), lr = self.lr)
        elif optimizer == 'adam':
            self.encoder_optimizer = optim.Adam(self.encoder.parameters(), lr = self.lr)
            self.decoder_optimizer = optim.Adam(self.decoder.parameters(), lr = self.lr)
        else:
            raise ValueError('Invalid optimizer specified')
        
        # Initialize Criterion
        self.criterion = nn.NLLLoss()

    def train(self, input_tensor, target_tensor):

        # Initialize Encoder Hidden State
        encoder_hidden = self.encoder.initHidden()
        encoder_cell = self.encoder.initHidden()
        encoder_outputs = torch.zeros(MAX_LEN, self.encoder.hidden_size, device=device)

        # Zero Gradients
        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()

        # Get input and target length
        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)

        # Initialize Loss
        loss = 0

        # Forward pass through the encoder
        for ei in range(input_length):
            encoder_output, encoder_hidden, encoder_cell = self.encoder(input_tensor[ei], encoder_hidden, encoder_cell)
            encoder_outputs[ei] = encoder_output[0, 0]

        # Initialize Decoder Hidden State
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden, decoder_cell = encoder_hidden, encoder_cell

        # Use Teacher Forcing
        use_teacher_forcing = True if random.random() < 0.5 else False
        
        # Forward pass through the decoder
        if use_teacher_forcing:
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_cell = self.decoder(decoder_input, decoder_hidden, decoder_cell)
                loss += self.criterion(decoder_output, target_tensor[di])
                decoder_input = target_tensor[di]
        else:
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_cell = self.decoder(decoder_input, decoder_hidden, decoder_cell)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()
                loss += self.criterion(decoder_output, target_tensor[di])
                if decoder_input.item() == EOS_token:
                    break

        # Backpropagate Loss
        loss.backward()

        # Update Parameters
        self.encoder_optimizer.step()
        self.decoder_optimizer.step()

        return loss.item() / target_length
    
    def evaluate(self, word: str):
        with torch.no_grad():
            input_tensor = tensorFromWord(self.input_lang, word)
            input_length = input_tensor.size()[0]
            encoder_hidden = self.encoder.initHidden()
            encoder_cell = self.encoder.initHidden()

            encoder_outputs = torch.zeros(MAX_LEN, self.encoder.config.hidden_size, device=device)

            for ei in range(input_length):
                encoder_output, encoder_hidden, encoder_cell = self.encoder(input_tensor[ei],
                                                        encoder_hidden, encoder_cell)
                encoder_outputs[ei] += encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

            decoder_hidden, decoder_cell = encoder_hidden, encoder_cell

            decoded_words = []

            for di in range(MAX_LEN):
                decoder_output, decoder_hidden, decoder_cell = self.decoder(
                    decoder_input, decoder_hidden, decoder_cell)
                topv, topi = decoder_output.data.topk(1)
                if topi.item() == EOS_token:
                    decoded_words.append('<EOS>')
                    break
                else:
                    decoded_words.append(self.output_lang.index2letter[topi.item()])

                decoder_input = topi.squeeze().detach()

            return decoded_words
        
    def accuracy(self, pairs):
        
        # Returns the accuracy of the model on the given pairs
        correct = 0
        total = 0
        for pair in pairs:
            output_words = self.evaluate(pair[0])
            output_sentence = ''.join(output_words)
            if output_sentence == pair[1]:
                correct += 1
            total += 1
        return correct / total
        
    def trainEpoch(self, epochs = 5, Log = False):
        
        # Initializations
        start = time.time()
        plot_losses = []
        print_loss_total = 0  # Reset every 1000 steps
        plot_loss_total = 0  # Reset every 100
        prev_train_acc = 0
        prev_val_acc = 0

        # Train for given number of epochs
        for epoch in range(1, epochs + 1):
            print('Epoch: ', epoch)

            # Get Tensor Pairs
            training_pairs = [tensorFromPair(pair) for pair in train_pairs]

            # Train for one epoch
            for i in range(1, len(training_pairs) + 1):
                training_pair = training_pairs[i - 1]
                input_tensor = training_pair[0]
                target_tensor = training_pair[1]

                loss = self.train(input_tensor, target_tensor)
                print_loss_total += loss
                plot_loss_total += loss

                # Print Progress
                if i % 1000 == 0:
                    temp = len(train_pairs)
                    print_loss_avg = print_loss_total / 1000
                    print_loss_total = 0
                    print('%s (%d %d%%) %.4f' % (timeSince(start, i / len(train_pairs)),
                                                i, i /  len(train_pairs) * 100, print_loss_avg))

                # Plot progess
                if i % 100 == 0:
                    plot_loss_avg = plot_loss_total / 100
                    plot_losses.append(plot_loss_avg)
                    plot_loss_total = 0
                    
            # Training Loss
            train_loss = sum(plot_losses) / len(plot_losses)

            # Train Accuracy
            train_acc = self.accuracy(train_pairs)
            print('Train Accuracy: ', train_acc)

            # Validation Accuracy
            val_acc = self.accuracy(valid_pairs)

            # Check to end
            if val_acc < prev_val_acc or train_acc < prev_train_acc:
                break
                
            # Update previous accuracy
            prev_train_acc = train_acc
            prev_val_acc = val_acc

        # Test Accuracy
        test_acc = self.accuracy(test_pairs)
        print('Test Accuracy: ', test_acc)



In [108]:
model = Seq2Seq(hidden_size = 256)
model.trainEpoch(epochs = 5)

/home/rohan/.local/lib/python3.11/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch:  1
0m 20s (- 17m 21s) (1000 1%) 28.4639
0m 43s (- 17m 45s) (2000 3%) 27.4905
1m 1s (- 16m 27s) (3000 5%) 27.4846


KeyboardInterrupt: 